## markdown 파일에 따르면, 아래의 코드를 실행해야 함 (상세 Argument는 제외)
```bash
cd nasbench201 && bash darts-201.sh
```
⇒ [darts-201.sh](http://darts-201.sh) 파일을 보니, train_serach.py를 실행함

## train_search.py

```
main : nasbench201.train.py

sub1 : nasbench201.cell_operations.SearchSpaceNames => 하단 설명
sub2 : nasbench201.search_model_darts.TinyNetworkDarts => 다른 파일에서 설명
sub3 : nasbench201.search_model_darts_proj.TinyNetworkDartsProj => 다른 파일에서 설명
sub4 : nasbench201.architect_ig.Architect => 하단 설명
```

### Arguments
- `--data`: 데이터셋의 위치
- `--dataset` : 데이터셋의 이름, cifar10, cifar100, ImageNet16-120 중 하나
- `--method`: NAS 방법, dirichlet
- `--search_space` : search space의 이름, nas-bench-201
- `--batch_size` : batch size, default : 64
- `--learning_rate` : learning rate, default : 0.025
- `--learning_rate_min` : minimum learning rate, default : 0.001
- `--momentum` : momentum, default : 0.9
- `--weight_decay` : weight decay, default : 3e-4
- `--report_freq` : report frequency, default : 50
- `--gpu` : gpu id, default : 'auto'
- `--epochs` : epochs, default : 100
- `--init_channels` : initial channels, default : 16
- `--layers` : layers, default : 8
- `--model_path` : model path, default : 'saved_models'
- `--cutout` : cutout, default : False
- `--cutout_length` : cutout length, default : 16
- `--cutout_prob` : cutout probability, default : 1.0
- `--save` : save, default : 'exp'
- `--seed` : seed, default : 2
- `--grad_clip` : grad clip, default : 5
- `--train_portion` : train portion, default : 0.5
- `--arch_learning_rate` : arch learning rate, default : 3e-4
- `--arch_weight_decay` : arch weight decay, default : 1e-3
- `--fast` : 사전 학습된 모델 구조를 가져옴, default : False
- `--resume_epoch` : resume epoch, default : 0
- `--resume_expid` : resume expid, default : ''
- `--dev` : dev, default : None
- `--ckpt_interval` : ckpt interval, default : 20
- `--expid_tag` : expid tag, default : 'none'
- `--log_tag` : log tag, default : ''
- `--edge_decision` : edge decision, default : 'random'
- `--proj_crit` : proj crit, default : 'acc'
- `--proj_intv` : proj intv, default : 5

In [ ]:
if args.dataset == 'cifar100': n_classes = 100
elif args.dataset == 'imagenet16-120': n_classes = 120
else: n_classes = 10

line 1 : cifar100 : 100개의 클래스<br>
line 2 : imagenet16-120 : 120개의 클래스<br>
line 3 : cifar10 : 10개의 클래스

In [ ]:
torch.set_num_threads(3)
if not torch.cuda.is_available():
    logging.info('no gpu device available')
    sys.exit(1)

np.random.seed(args.seed)
gpu = ig_utils.pick_gpu_lowest_memory() if args.gpu == 'auto' else int(args.gpu)
torch.cuda.set_device(gpu)
cudnn.benchmark = True
torch.manual_seed(args.seed)
cudnn.enabled = True
torch.cuda.manual_seed(args.seed)

line 1 : 3개의 thread 사용<br>
line 2~4 : GPU가 불가능하면 PASS<br>
line 6 : seed 고정<br>
line 7 : GPU 사용(auto면 GPU가 제일 적게 쓰이고 있는 GPU를 선택)<br>
line 8 : GPU 세팅<br>
line 9~12 : Reproducibility를 위한 세팅

In [ ]:
if not args.fast:
    api = API('../data/NAS-Bench-201-v1_0-e61699.pth')

line 1~2 : 사전학습된 모델 Architecture

In [ ]:
criterion = nn.CrossEntropyLoss()
search_space = SearchSpaceNames[args.search_space]
if args.method in ['darts', 'blank']:
    model = TinyNetworkDarts(C=args.init_channels, N=5, max_nodes=4, num_classes=n_classes, criterion=criterion, search_space=search_space, args=args)
elif args.method in ['darts-proj', 'blank-proj']:
    model = TinyNetworkDartsProj(C=args.init_channels, N=5, max_nodes=4, num_classes=n_classes, criterion=criterion, search_space=search_space, args=args)
model = model.cuda()

architect = Architect(model, args)

line 1 : creiterion : nn.CrossEntropy()<br>
line 2 : search_space : 하단 참조
```
CONNECT_NAS_BENCHMARK = ['none', 'skip_connect', 'nor_conv_3x3']
NAS_BENCH_201         = ['none', 'skip_connect', 'nor_conv_1x1', 'nor_conv_3x3', 'avg_pool_3x3']
DARTS_SPACE           = ['none', 'skip_connect', 'dua_sepc_3x3', 'dua_sepc_5x5', 'dil_sepc_3x3', 'dil_sepc_5x5', 'avg_pool_3x3', 'max_pool_3x3']
NAS_BENCH_201_SKIP    = ['none', 'skip_connect', 'nor_conv_1x1_skip', 'nor_conv_3x3_skip', 'avg_pool_3x3']
NAS_BENCH_201_SIMPLE  = ['skip_connect', 'nor_conv_1x1', 'nor_conv_3x3', 'avg_pool_3x3']
NAS_BENCH_201_S2      = ['skip_connect', 'nor_conv_3x3']
NAS_BENCH_201_S4      = ['noise', 'nor_conv_3x3']
NAS_BENCH_201_S10     = ['none', 'nor_conv_3x3']
```
line 3~4 : TinyNetworkDarts for DARTS, Blank ( 다른 파일 추가 설명 참조 )<br>
line 5~6 : TinyNetworkDartsProj for DARTS-proj, Blank-proj ( 다른 파일 추가 설명 참조 )<br>
line 7 : model to gpu<br>
line 9 : Architecture에 포함 ( 다음 Cell 추가 설명 참조 )

In [ ]:
class Architect(object):
    def __init__(self, model, args):
        self.network_momentum = args.momentum
        self.network_weight_decay = args.weight_decay
        self.model = model
        self.optimizer = torch.optim.Adam(self.model.arch_parameters(),
                                        lr=args.arch_learning_rate, betas=(0.5, 0.999),
                                        weight_decay=args.arch_weight_decay)
        self._init_arch_parameters = []
        for alpha in self.model.arch_parameters():
            alpha_init = torch.zeros_like(alpha)
            alpha_init.data.copy_(alpha)
            self._init_arch_parameters.append(alpha_init)
        if args.method in ['darts', 'darts-proj', 'sdarts', 'sdarts-proj']:
            self.method = 'fo' # first order update
        elif 'so' in args.method:
            print('ERROR: PLEASE USE architect.py for second order darts')
        elif args.method in ['blank', 'blank-proj']:
            self.method = 'blank'
        else:
            print('ERROR: WRONG ARCH UPDATE METHOD', args.method); exit(0)
    def reset_arch_parameters(self):
        for alpha, alpha_init in zip(self.model.arch_parameters(), self._init_arch_parameters):
            alpha.data.copy_(alpha_init.data)
    def step(self, input_train, target_train, input_valid, target_valid, *args, **kwargs):
        if self.method == 'fo':
            shared = self._step_fo(input_train, target_train, input_valid, target_valid)
        elif self.method == 'so':
            raise NotImplementedError
        elif self.method == 'blank': ## do not update alpha
            shared = None
        return shared
    #### first order
    def _step_fo(self, input_train, target_train, input_valid, target_valid):
        loss = self.model._loss(input_valid, target_valid)
        loss.backward()
        self.optimizer.step()
        return None
    #### darts 2nd order
    def _step_darts_so(self, input_train, target_train, input_valid, target_valid, eta, model_optimizer):
        raise NotImplementedError

line 1 : Architecture<br>
line 2~21 : Initialization<br>
- line 3 : network_momenum : argument에 있는 momentum
- line 4 : network_weight_decay : argument에 있는 weight decay
- line 5 : model : model
- line 6~8 : Optimizer 세팅
    - line 6 : self.model.arch_paramters : 모델 파라미터
    - line 7 : lr : argument에 있는 arch learning rate, betas : (0.5, 0.999)
    - line 8 : weight_decay : argument에 있는 arch weight decay
- line 9~13 : 모델 weight를 0으로 만들었을 때의 weight를 self._init_arch_parameters에 저장
- line 14~21 : self.method 등록
    - line 14~15 : DARTS, DARTS-proj, SDARTS, SDARTS-proj이면 self.proj = 'fo' : first order
    - line 16~17 : 'so' in arg.method : second order, error
    - line 18~19 : BLANK, BLANK-PROJ =  'blank'
    - line 20~21 : error<br>

line 22~24 : reset_arch_parameters : 모델 파라미터를 0으로 초기화<br>
line 25~32 : step : 모델 업데이트
- line 25 : 함수 정의
- line 26~27 : first order들 업데이트
    - shared = self._step_fo
- line 28~29 : second order들 업데이트
    - not implemented
- line 30~31 : blank들 업데이트
    - shared = None
- line 32 : `return shared`<br>

line 34~38 : first order 업데이트
- line 35 : loss 계산
- line 36 : loss 역전파
- line 37 : optimizer step
- return None

line 40~41 : second order : Not Implemented Error

In [ ]:
if args.dataset == 'cifar10':
    train_transform, valid_transform = ig_utils._data_transforms_cifar10(args)
    train_data = dset.CIFAR10(root=args.data, train=True, download=True, transform=train_transform)
    valid_data = dset.CIFAR10(root=args.data, train=False, download=True, transform=valid_transform)
elif args.dataset == 'cifar100':
    train_transform, valid_transform = ig_utils._data_transforms_cifar100(args)
    train_data = dset.CIFAR100(root=args.data, train=True, download=True, transform=train_transform)
    valid_data = dset.CIFAR100(root=args.data, train=False, download=True, transform=valid_transform)
elif args.dataset == 'imagenet16-120':
    import torchvision.transforms as transforms
    from nasbench201.DownsampledImageNet import ImageNet16
    mean = [x / 255 for x in [122.68, 116.66, 104.01]]
    std = [x / 255 for x in [63.22,  61.26, 65.09]]
    lists = [transforms.RandomHorizontalFlip(), transforms.RandomCrop(16, padding=2), transforms.ToTensor(), transforms.Normalize(mean, std)]
    train_transform = transforms.Compose(lists)
    train_data = ImageNet16(root=os.path.join(args.data,'imagenet16'), train=True, transform=train_transform, use_num_of_class_only=120)
    valid_data = ImageNet16(root=os.path.join(args.data,'imagenet16'), train=False, transform=train_transform, use_num_of_class_only=120)
    assert len(train_data) == 151700

num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(args.train_portion * num_train))

train_queue = torch.utils.data.DataLoader(
    train_data, batch_size=args.batch_size,
    sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[:split]),
    pin_memory=True)
valid_queue = torch.utils.data.DataLoader(
    train_data, batch_size=args.batch_size,
    sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[split:num_train]),
    pin_memory=True)

line 1~4 : CIFAR10 Load Train, Test Datasets<br>
line 5~8 : CIFAR100 Load Train, Test Datasets<br>
line 9~18 : ImageNet16-120 Load Train, Test Datasets<br>
line 20 : 데이터 크기<br>
line 21 : indices<br>
line 22 : split size : TRAIN : VALID Split<br>
line 24~27 : Train DataLoader<br>
line 28~31 : Valid DataLoader

In [ ]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    model.optimizer, float(args.epochs), eta_min=args.learning_rate_min)

line 1~2 : Scheduler 세팅
- CosineAnnealingLR

In [ ]:
# Check Point 세팅이 되어있으면, 해당 세팅을 불러온다.
start_epoch = 0
if args.resume_epoch != 0:
    checkpoint = torch.load(filename, map_location='cpu')
    start_epoch = checkpoint['epoch'] # epoch
    model_state_dict = checkpoint['state_dict']
    if '_arch_parameters' in model_state_dict: del model_state_dict['_arch_parameters']
    model.load_state_dict(model_state_dict) # model
    saved_arch_parameters = checkpoint['alpha'] # arch
    model.set_arch_parameters(saved_arch_parameters)
    scheduler.load_state_dict(checkpoint['scheduler'])
    model.optimizer.load_state_dict(checkpoint['optimizer']) # optimizer
    architect.optimizer.load_state_dict(checkpoint['arch_optimizer'])

line 2~13 : 세팅 불러오기
- line 2 : start epoch
- line 3 : resume_epoch가 남아있을 경우 실행
- line 3~13 : 학습된 모델 불러오기

In [ ]:
for epoch in range(start_epoch, args.epochs):
    lr = scheduler.get_lr()[0]
    if args.cutout:
        train_transform.transforms[-1].cutout_prob = args.cutout_prob * epoch / (args.epochs - 
    genotype = model.genotype()
    train_acc, train_obj = train(train_queue, valid_queue, model, architect, model.optimizer, lr, epoch)
    valid_acc, valid_obj = infer(valid_queue, model, criterion, log=False)
    scheduler.step()
    save_state = {
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'alpha': model.arch_parameters(),
        'optimizer': model.optimizer.state_dict(),
        'arch_optimizer': architect.optimizer.state_dict(),
        'scheduler': scheduler.state_dict()
    }
    if save_state['epoch'] % args.ckpt_interval == 0:
        ig_utils.save_checkpoint(save_state, False, args.save, per_epoch=True)

line 1~18 : Train Model
- line 1 : for loop
- line 2 : get learning rate
- line 3~4 : change cutout prob
    - line 4 : increase cutout ratio while epoch increases (ex : 0.025 if epoch == 1 else 0.5 if epoch == 200, epochs = 200)
- line 5 : genotype 설정, TinyNetworkDarts 참조
- line 6 : train : 하단 참조
- line 7 : infer : 하단 참조
- line 8 : scheduler 업데이트
- line 9~16 : save 할 정보들 업데이트
- 17~18 : epoch가 저장할 때마다 저장


In [ ]:
def train(train_queue, valid_queue, model, architect, optimizer, lr, epoch):
    objs = ig_utils.AvgrageMeter()
    top1 = ig_utils.AvgrageMeter()
    top5 = ig_utils.AvgrageMeter()
    for step in range(len(train_queue)):
        model.train()
        input, target = next(iter(train_queue))
        input = input.cuda(); target = target.cuda(non_blocking=True)
        input_search, target_search = next(iter(valid_queue))
        input_search = input_search.cuda(); target_search = target_search.cuda(non_blocking=True)
        optimizer.zero_grad(); architect.optimizer.zero_grad()
        shared = architect.step(input, target, input_search, target_search,
                                eta=lr, network_optimizer=optimizer)
        optimizer.zero_grad(); architect.optimizer.zero_grad()
        logits, loss = model.step(input, target, args, shared=shared)
        prec1, prec5 = ig_utils.accuracy(logits, target, topk=(1, 5))
        n = input.size(0)
        objs.update(loss.data, n)
        top1.update(prec1.data, n)
        top5.update(prec5.data, n)
        if args.fast:
            break
    return top1.avg, objs.avg

line 1~23 : training code
- line 1 : function define
- line 2~4 : AvgrageMeter(), 하단 참조
- line 5 : Data Loader
- line 6 : make model training mode
- line 7 : load training input and target
- line 8 : input, target to gpu
- line 9 : load validation input and target
- line 11~13 : update model architecture alpha
    - line 11 : zero grad for model and architecture
    - line 12~13 : update architecture
- line 14~15 : update model weight
    - line 14 : zero grad for model and architecture
    - line 15 : update model weight
- line 16 : get accuracy
- line 17 : get batch size
- line 18~20 : update avgmeter
    - line 18 : update loss
    - line 19 : update top1 acc
    - line 20 : update top5 acc
- line 21~22 : break
    - line 21 : if args.fast : 사전 학습된 모델의 구조이면,
    - line 22 : break -> 학습된 모델임
- line 23 : top1.avg, objs.avg

In [ ]:
class AvgrageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.cnt = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.cnt += n
        self.avg = self.sum / self.cnt

line 3 : init <br>
line 4 : reset, set avg, sum, cnt<br>
line 11~14 : update, arguments : value, n=1<br>
- line 12 : sum all values<br>
- line 13 : sum all counts<br>
- line 14 : average of all values<br>

In [ ]:
def infer(valid_queue, model, criterion,
          log=True, eval=True, weights=None, double=False, bn_est=False):
    objs = ig_utils.AvgrageMeter()
    top1 = ig_utils.AvgrageMeter()
    top5 = ig_utils.AvgrageMeter()
    model.eval() if eval else model.train() # disable running stats for projection
    if bn_est:
        _data_loader = deepcopy(valid_queue)
        for step, (input, target) in enumerate(_data_loader):
            input = input.cuda()
            target = target.cuda(non_blocking=True)
            with torch.no_grad():
                logits = model(input)
        model.eval()
    with torch.no_grad():
        for step, (input, target) in enumerate(valid_queue):
            input = input.cuda()
            target = target.cuda(non_blocking=True)
            if double:
                input = input.double(); target = target.double()
            logits = model(input) if weights is None else model(input, weights=weights)
            loss = criterion(logits, target)
            prec1, prec5 = ig_utils.accuracy(logits, target, topk=(1, 5))
            n = input.size(0)
            objs.update(loss.data, n)
            top1.update(prec1.data, n)
            top5.update(prec5.data, n)
            if args.fast:
                break
    return top1.avg, objs.avg

상세 설명은 생략
- bn_est : validation data도 업데이트? => 사용 안함
- double : double precision (float64)